# LightlyTrain with Ultralytics

In this tutorial we will demonstrate how you can use Lightly**Train** to pretrain a YOLO11s model from [Ultralytics](https://github.com/ultralytics/ultralytics). To this end, we will first use the raw images (**no labels**) from the [PASCAL Visual Object Classes (VOC) dataset](http://host.robots.ox.ac.uk/pascal/VOC/) and pretrain for 100 epochs. After pretraining, we will show that we can fine-tune the model to the task of object dection and significantly outperform training from scratch in terms of *acccuracy* and *convergence speed*.

In order to get started, let's first make sure that:
 - We installed all required packages. 
 - We have the dataset downloaded and ready in the correct format.

# Installation

The required packages include `lightly-train` with support for `ultralytics`, these are the packages we need for pretraining and finetuning. 

As utilities, we'll also install Roboflow's `supervision`, which let's us easily visualize a few annotated data samples, as well as `ipywidgets`, which enhances a few interactive elements of this Jupyter notebook.

In [ ]:
!pip install "lightly-train[ultralytics]" "supervision==0.25.1" ipywidgets

> **Important**: LightlyTrain is officially supported on
> - Linux: CPU or CUDA
> - MacOS: CPU only
> - Windows (experimental): CPU or CUDA
> 
> We are planning to support MPS for MacOS.
> 
> Check the [installation instructions](https://docs.lightly.ai/train/stable/installation.html) for more details on installation.

## Dataset Preprocessing
We can directly use Ultralytics' `check_det_dataset` function to download the VOC dataset.

In [ ]:
from ultralytics.data.utils import check_det_dataset

dataset = check_det_dataset("VOC.yaml")

Ultralytics always uses a fixed directory to save your datasets and you can fetch the location through their `settings` module:

In [ ]:
from ultralytics import settings

settings["datasets_dir"]

Our dataset directory is now ready under the path from above and will have the following structure:

```bash
datasets/VOC
├── images
│   ├── test2007
│   ├── train2007
│   ├── train2012
│   ├── val2007
│   └── val2012
└── labels
    ├── test2007
    ├── train2007
    ├── train2012
    ├── val2007
    └── val2012
```

## Dataset Inspection
Before we start, let's quickly also look at a few images of the training set, together with their annotations.

In [ ]:
import random

import matplotlib.pyplot as plt
import supervision as sv
import yaml

detections = sv.DetectionDataset.from_yolo(
    data_yaml_path=dataset["yaml_file"],
    images_directory_path=f"{settings['datasets_dir']}/VOC/images/train2012",
    annotations_directory_path=f"{settings['datasets_dir']}/VOC/labels/train2012",
)

with open(dataset["yaml_file"], "r") as f:
    data = yaml.safe_load(f)
names = data["names"]

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

fig, ax = plt.subplots(2, 2, figsize=(10, 10))
ax = ax.flatten()

detections = [detections[random.randint(0, len(detections))] for _ in range(4)]

for i, (path, image, annotation) in enumerate(detections):
    annotated_image = box_annotator.annotate(scene=image, detections=annotation)
    annotated_image = label_annotator.annotate(
        scene=annotated_image,
        detections=annotation,
        labels=[names[elem] for elem in annotation.class_id],
    )
    ax[i].imshow(annotated_image[..., ::-1])
    ax[i].axis("off")

## Pretraining with Lightly**Train**
Pretraining with Lightly**Train** could not be easier, you just pass the following parameters: 
 - `out`: you simply state where you want your logs and exported model to go to
 - `model`: the model that you want to train, e.g. `yolo11s` from Ultralytics
 - `data`: the path to a folder with images

 Your data is simply assumed to be an arbitrarily nested folder; LightlyTrain with find all images on its own and since there are no labels required there is no danger of ever using false labels! 🕵️‍♂️

It is highly recommended to run this pretraining on a GPU, expect about 60min of training time on Colab's free version!

In [ ]:
import lightly_train

lightly_train.train(
    out="logs/pretrain_yolo11s",
    data=f"{settings['datasets_dir']}/VOC/images/train2012",
    model="ultralytics/yolo11s.yaml",
    overwrite=True,  # we allow overwriting so that you can conveniently run this cell repeatedly
)

## Fine-tuning with Ultralytics

Now that the pretrained model has been exported, we will further fine-tune the model on the task of object detection. The exported model already has exactly the format that Ultralytics' YOLO expects, so we can get started with only a few lines! ⚡️ In addition to fine-tuning the pretrained model we will also train a model that we initialize with random weights. This will let us compare the performance between the two, and show the great benefits of pretraining.

Expect again a run-time of around 1h each for fine-tuning from the pretrained model as well as fine-tuning from randomly initialized weights.

In [ ]:
from ultralytics import YOLO

model_pretrained = YOLO("logs/pretrain_yolo11s/exported_models/exported_last.pt")
model_pretrained.train(
    data="VOC.yaml",
    epochs=30,
    project="logs/voc_yolo11s",
    name="from_pretrained",
)

In addition to fine-tuning the pretrained model we will also train a model that we initialize with random weights. This will let us compare the performance between the two, and show the great benefits of pretraining. ⚡️

In [ ]:
model_scratch = YOLO("yolo11s.yaml")
model_scratch.train(
    data="VOC.yaml",
    epochs=30,
    project="logs/voc_yolo11s",
    name="from_scratch",
)

## Performance Evaluation
Congratulations, you made it almost to the end! 🎉 The last thing we'll do is to analyze the performance between the two. A very common metric to measure the performance of object detectors is the `mAP50-95` which we plot in the next cell, for both the pretrained model and the model that we trained from scratch.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

res_scratch = pd.read_csv("logs/voc_yolo11s/from_scratch/results.csv")
res_finetune = pd.read_csv("logs/voc_yolo11s/from_pretrained/results.csv")

fig, ax = plt.subplots()
ax.plot(res_scratch["epoch"], res_scratch["metrics/mAP50-95(B)"], label="scratch")
ax.plot(res_finetune["epoch"], res_finetune["metrics/mAP50-95(B)"], label="finetune")
ax.set_xlabel("Epoch")
ax.set_ylabel("mAP50-95")
max_pretrained = res_finetune["metrics/mAP50-95(B)"].max()
max_scratch = res_scratch["metrics/mAP50-95(B)"].max()
ax.set_title(
    f"Pretraining is {(max_pretrained - max_scratch) / max_scratch * 100:.2f}% better than scratch"
)
ax.legend()

As you can see, the pretrained model outperforms the model trained from scratch by a significant margin, thanks to the magic of Lightly**Train**. Now you are ready to leverage it to pretrain on your own data!